In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [2]:
encpb="../model/Trained_Graphs/encoder_frozen_model.pb"

In [3]:
with open(encpb, 'rb') as f:
    fileContent = f.read()
graph_def = tf.GraphDef()
graph_def.ParseFromString(fileContent)
output1="Output_Features:0"

In [4]:
output1 = tf.import_graph_def(
    graph_def,
    input_map=None,
    return_elements=[output1],
    name='encoder')
graph = tf.get_default_graph()

In [5]:
output1

[<tf.Tensor 'encoder/Output_Features:0' shape=(1, 1536) dtype=float32>]

In [6]:
decpb = "../model/Trained_Graphs/decoder_frozen_model.pb"

In [7]:
with open(decpb, 'rb') as f:
    fileContent = f.read()
graph_def = tf.GraphDef()
graph_def.ParseFromString(fileContent)
input2="Input_Features:0"

In [8]:
tf.import_graph_def(
    graph_def,
    input_map={input2:output1[0]},
    return_elements=None,
    name='decoder')
graph = tf.get_default_graph()

In [ ]:
#output_node_names = [
#        "Preprocessed_JPG",
#        "Preprocessed_PNG",
#        "OutputFeatures"]

In [13]:
with open("../model/Decoder/DecoderOutputs.txt", 'r') as f:
    output = f.read()
    prefix = ""
    outputs=[]
    outputs += [prefix + o for o in output.split('\n')[:-1]]
outputs

['LSTM/word_0',
 'LSTM/word_1',
 'LSTM/word_2',
 'LSTM/word_3',
 'LSTM/word_4',
 'LSTM/word_5',
 'LSTM/word_6',
 'LSTM/word_7',
 'LSTM/word_8',
 'LSTM/word_9',
 'LSTM/word_10',
 'LSTM/word_11',
 'LSTM/word_12',
 'LSTM/word_13',
 'LSTM/word_14',
 'LSTM/word_15',
 'LSTM/word_16',
 'LSTM/word_17',
 'LSTM/word_18',
 'LSTM/word_19',
 'LSTM/word_20',
 'LSTM/word_21']

In [19]:
import numpy as np
wtoidx = np.load("../Dataset/wordmap.npy").tolist()
idxtow = dict(zip(wtoidx.values(), wtoidx.keys()))
def IDs_to_Words(ID_batch):
    return [idxtow[word] for IDs in ID_batch for word in IDs]

In [20]:
with tf.Session() as sess:
    in1 = graph.get_tensor_by_name("encoder/InputFile:0")
    out1 = graph.get_tensor_by_name("encoder/Preprocessed_JPG:0")
    feed_dict = {in1: "../Images/3.jpg"}
    prepro_image = sess.run(out1, feed_dict=feed_dict)
    in2 = graph.get_tensor_by_name("encoder/import/InputImage:0")
    sentence = []
    for i,outs in enumerate(outputs):
        sentence.append(graph.get_tensor_by_name("decoder/"+outs+":0"))
    feed_dict = {in2:prepro_image}
    prob= sess.run(sentence, feed_dict=feed_dict)
    print " ".join(IDs_to_Words(prob)).split("</S>")[0]

a man in a blue shirt and a woman in a white shirt are walking down a dirt road 
